In [1]:
import json

In [2]:
dataset = '/home/nghia/data/datasets/news-clustering/news-aggregator.jsonl'

In [3]:
articles = []
with open(dataset) as in_f:
    for line in in_f:
        articles.append(json.loads(line))

In [4]:
articles[0]

{'id': '173760',
 'title': 'Shell boss touts “robust” profits despite drop',
 'body': 'Shell boss touts “robust” profits despite drop\nAnglo-Dutch oil giant Shell touted “robust” profits for the first three months of 2014 today, despite a nearly halving in size compared with the start of last year.\n\nFirst quarter earnings in 2014 were $4.5 billion (£2.7bn) compared with $8 billion (£4.7bn) for the first quarter of 2013.\n\nChief Executive Ben van Beurden said: “Our first quarter 2014 results reflect more robust levels of profitability. However, as we saw in 2013, we are in an industry where high volatility remains, both in the macro-environment and in our quarterly results.”\n\nHe said the year has seen new, profitable production from the deep-water Gulf of Mexico and Iraq along with new LNG from our its new buy from Repsol.\n\nExpressing determination to improve Shell’s competitiveness, Mr van Beurden said his firm would balance growth and returns with “better financial performance”

In [5]:
from aylienapiclient import textapi
client = textapi.Client("6f0c8b31", "a6198a07f07f3e4de801137a87ea9c7b")

In [26]:
def clean_sentence(sentence):
    return sentence.replace('\n',' <q> ').replace('\r',' <q> ')

def process_article(art, client):
    data = {}
    if 'title' in art and art['title'] != '':
        data['title'] = art['title']
    if 'body' in art and art['body'] != '':
        data['text'] = art['body']
    for i in range(3):
        try:
            summary_sentences = client.Summarize(data)['sentences']
            break
        except:
            if i == 2:
                print('Tried 3 times, return empty string')
                return ''
            print('Cannot get summary, trying again')
            time.sleep(1)
    return ' <q> '.join([clean_sentence(sentence) for sentence in summary_sentences])
    

In [10]:
process_article(articles[0], client)

'Shell boss touts “robust” profits despite drop <q> Anglo-Dutch oil giant Shell touted “robust” profits for the first three months of 2014 today, despite a nearly halving in size compared with the start of last year. <q> First quarter earnings in 2014 were $4.5 billion (£2.7bn) compared with $8 billion (£4.7bn) for the first quarter of 2013. <q> Chief Executive Ben van Beurden said: “Our first quarter 2014 results reflect more robust levels of profitability. <q> He said the year has seen new, profitable production from the deep-water Gulf of Mexico and Iraq along with new LNG from our its new buy from Repsol. <q> Expressing determination to improve Shell’s competitiveness, Mr van Beurden said his firm would balance growth and returns with “better financial performance”, picking projects “selectively” and divesting $15 billion of assets between 2014-15.'

In [27]:
def create_eval_data(articles, client, k):
    num = 0
    success = 0
    good_arts = []
    summaries = []
    for art in articles:
        summary = process_article(art, client).strip()
        if summary != '':
            good_arts.append(art)
            summaries.append(summary)
            success += 1
        if success % 20 == 0:
            print(success)
        if success == k:
            break
    return good_arts, summaries

In [28]:
good_arts, summaries = create_eval_data(articles, client, 500)

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
480
500


In [29]:
def print_lines(lines, output_file):
    with open(output_file, 'w') as out_f:
        for line in lines:
            out_f.write(f'{line}\n')

In [30]:
output_dir = '/home/nghia/data/datasets/summarization/news-aggregator-top-n'
text_rank_file = output_dir + '/textrank/news-aggregator-500.textrank.cand'

In [31]:
print_lines(summaries, text_rank_file)

In [32]:
news_format_file = output_dir + '/news/news-aggregator-500.jsonl'

In [33]:
with open(news_format_file, 'w') as out_f:
    for art in good_arts:
        out_f.write(f'{json.dumps(art)}\n')